## Librerías

In [20]:
import ee
import json

## Autenticación

Primeros autenticarnos con nuestras credenciales, para esto necesitamos registrarnos.

In [2]:
ee.Authenticate()


Successfully saved authorization token.


## Inicialización de la librería

In [3]:
ee.Initialize()

## Delimitación de la zona/zonas de estudio

Para esto vamos a obtener la geometría desde un archivo geojson.

In [8]:
with open(r'C:\Users\DELL\PycharmProjects\Tesis\Poligonos\Rancho_SantaAnita\Rancho-Santa-Anita.geojson', encoding='utf-8') as geom:
    rancho = json.load(geom)

## Pasamos este json a un FeatureCollection

Esto para poder operar las geometrías con las colecciones de imágenes.

In [10]:
fc = ee.FeatureCollection(rancho)

Vamos a calcular el área para ver que coincida con las dimensiones esperadas.

In [14]:
area = fc.geometry().area()
print(f'El área es de {area.divide(10000).getInfo()} ha.')

El área es de 101.06964064167524 ha.


# Colección de imágenes

## Misión Sentinel 2

La colección es de las imágenes con nivel de procesamiento 2A.

In [69]:
start_year = '2021-01-01'
end_year = '2023-09-01'

In [70]:
ic = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterDate(start=start_year, opt_end=end_year).filter(ee.Filter.eq('MGRS_TILE', '14QML'))

In [71]:
ic.size().getInfo()

402

En la celda anterior se definió la colección de imágenes con un rango de fechas determinado y que solo sean las que coinciden con el **TILE_ID** = 14QML, que abarca la región de interés.

### AOI dentro de la imagen

Vamos a asegurarnos que el área de interés se encuentre dentro de la imagen, para esto se comparan las geometrías de ambos elementos.

In [38]:
def does_intersect(img, feature_geom):
    
    intersecta = img.geometry().intersects(feature_geom)
    
    return intersecta.getInfo()


### Filtro de nubes

En este apartado vamos a filtrar las imágenes que tienen nubes en la zona de estudio, las que presenten nubosidad en el área serán descartadas.

In [51]:
def count_cloud_pixels(img, cloud_percent, aoi):
    
    cloud_band = img.select('MSK_CLDPRB').gt(cloud_percent)
    pixel_count = cloud_band.reduceRegion(
        reducer = ee.Reducer.frequencyHistogram(),
        geometry = aoi,
        scale = 10,
        maxPixels = 1e9
    )
    
    return pixel_count.get('MSK_CLDPRB').getInfo()

## Exportación de imágenes a Google Drive

Teniendo los **id's** procedemos a exportar las imágenes a la carpeta de nuestra preferencia y con los parámetros deseados.

In [67]:
def export_images(folder: str, img, bands, aoi, to_crs: str, filename):
    
    raw_image = img.select(bands)
    
    export_params = {
        'driveFolder' : f'{folder}',
        'fileFormat': 'GeoTiff',
        'scale': 10,
        'region': aoi,
        'crs': to_crs
    }
    
    task = ee.batch.Export.image.toDrive(image=raw_image, description=filename, **export_params)
    
    task.start()
    task.status()
    
    return print(f"{filename} exportada correctamente.")

In [72]:
inside = 0
out = 0
cloud_free = 0

for image in ic.getInfo()['features']:
    resultado = does_intersect(img=ee.Image(image['id']), feature_geom=fc)
    #print(resultado)
    if resultado:
        #print(f"La imagen {image['id']} intersecta")
        counted_pixels = count_cloud_pixels(img=ee.Image(image['id']), cloud_percent=50, aoi=fc)
        #print(counted_pixels)
        if '1' not in counted_pixels:
            print(f"{image['id']} está completamente libre de nubes")
            export_images(folder='Tesis_caña', img=ee.Image(image['id']),bands=['B2', 'B3', 'B4', 'B8', 'B11', 'B12'], aoi=fc.geometry(), to_crs='EPSG:32614', filename=image['id'].replace('/', '_'))
            cloud_free += 1
        #free_pixels, cloud_pixels = counted_pixels['0'], counted_pixels['1']
        #print(free_pixels, cloud_pixels)
        inside += 1
    else:
        #print(f"La imagen {image['id']} no coincide")
        out += 1
        
print(f'Dentro: {inside}, No coincide: {out}, Libres de nubes: {cloud_free}')

COPERNICUS/S2_SR_HARMONIZED/20210106T170711_20210106T171413_T14QML está completamente libre de nubes
COPERNICUS_S2_SR_HARMONIZED_20210106T170711_20210106T171413_T14QML exportada correctamente.
COPERNICUS/S2_SR_HARMONIZED/20210116T170641_20210116T171345_T14QML está completamente libre de nubes
COPERNICUS_S2_SR_HARMONIZED_20210116T170641_20210116T171345_T14QML exportada correctamente.
COPERNICUS/S2_SR_HARMONIZED/20210126T170601_20210126T171307_T14QML está completamente libre de nubes
COPERNICUS_S2_SR_HARMONIZED_20210126T170601_20210126T171307_T14QML exportada correctamente.
COPERNICUS/S2_SR_HARMONIZED/20210131T170529_20210131T171614_T14QML está completamente libre de nubes
COPERNICUS_S2_SR_HARMONIZED_20210131T170529_20210131T171614_T14QML exportada correctamente.
COPERNICUS/S2_SR_HARMONIZED/20210205T170501_20210205T171201_T14QML está completamente libre de nubes
COPERNICUS_S2_SR_HARMONIZED_20210205T170501_20210205T171201_T14QML exportada correctamente.
COPERNICUS/S2_SR_HARMONIZED/2021021